In [1]:
from rdflib import Graph, URIRef, Literal, RDFS, OWL, RDF
from fuzzywuzzy import process
from owlready2 import *

In [2]:
def load_ontology(ontology_file):
    """
    Load the ontology from the given file.
    """
    graph = Graph()
    graph.parse(ontology_file, format='xml')  # Assuming OWL/XML format
    return graph

def extract_classes(ontology):
    """
    Extract all classes in the ontology as a list of dictionaries, including subclasses.
    """
    classes = []
    for cls in ontology.classes():
        class_info = {'name': cls.name}
        class_info.update(extract_metadata(cls))
        # Add subclasses
        class_info['subclasses'] = [sub.name for sub in cls.subclasses()]
        classes.append(class_info)
    return classes

def extract_metadata(entity):
    """
    Extract metadata for a given OWL entity (e.g., comment, label, defined_by).
    """
    metadata = {}
    if hasattr(entity, 'comment'):
        metadata['comment'] = entity.comment
    if hasattr(entity, 'label'):
        metadata['label'] = entity.label
    if hasattr(entity, 'isDefinedBy'):
        metadata['defined_by'] = [str(x) for x in entity.isDefinedBy]
    if hasattr(entity, 'iri'):
        metadata['iri'] = entity.iri
    return metadata

def extract_concepts(graph):
    """
    Extract all entities (classes/concepts) from the ontology.
    Returns a dictionary where keys are concept names and values are their parent concepts.
    """
    concepts = {}

    # Iterate over all OWL classes in the ontology
    for class_uri in graph.subjects(predicate=RDF.type, object=OWL.Class):
        # Get the label (name) of the class
        class_label = graph.value(subject=class_uri, predicate=RDFS.label)
        if class_label and isinstance(class_label, Literal):
            class_name = str(class_label)
        else:
            # If no label is found, use the local name of the URI
            class_name = str(class_uri.split("#")[-1])

        # Find parent classes (superclasses)
        parents = set()
        for parent_uri in graph.objects(subject=class_uri, predicate=RDFS.subClassOf):
            if isinstance(parent_uri, URIRef):
                parent_label = graph.value(subject=parent_uri, predicate=RDFS.label)
                if parent_label and isinstance(parent_label, Literal):
                    parents.add(str(parent_label))
                else:
                    parents.add(str(parent_uri.split("#")[-1]))

        concepts[class_name] = parents

    return concepts

def find_closest_matches(attribute, concepts, threshold=80):
    """
    Find all matches for the attribute in the list of concepts that exceed the threshold.
    Returns a list of matched concept dictionaries along with their scores.
    """
    # Extract concept names for fuzzy matching
    concept_names = [concept["name"] for concept in concepts]

    # Find all matches that exceed the threshold
    matches = process.extract(attribute, concept_names, limit=None)

    # Filter matches based on the threshold and return the full concept dictionaries with scores
    matched_concepts = []
    for match, score in matches:
        if score >= threshold:
            # Find the corresponding concept dictionary
            matched_concept = next((concept for concept in concepts if concept["name"] == match), None)
            if matched_concept:
                matched_concepts.append({"concept": matched_concept, "score": score})

    return matched_concepts

def select_parent_concept(matched_concepts):
    """
    Select the most specific concept (the one with the fewest subclasses) from the matches.
    """
    if not matched_concepts:
        return None

    # Sort matches by specificity (number of subclasses)
    sorted_matches = sorted(matched_concepts, key=lambda x: len(x.get("subclasses", [])))

    # Return the most specific concept (the one with the fewest subclasses)
    return sorted_matches[0]

def map_attributes_to_ontology(cram_action, ontology_file, threshold=80):
    """
    Map the attributes of the CRAM action designator to the ontology concepts.
    Returns a dictionary where each attribute is mapped to:
    - The closest match (most specific concept).
    - A list of all matches that exceed the threshold.
    """
    # Load the ontology
    # graph = load_ontology(ontology_file)
    onto = get_ontology(ontology_file).load()

    # Extract all entities (classes/concepts) and their hierarchy
    # concepts = extract_concepts(graph)
    concepts = extract_classes(onto)

    # Iterate over the CRAM action designator attributes
    mapped_action = {}
    for attr, value in cram_action.items():
        # Find all matches that exceed the threshold
        matched_concepts = find_closest_matches(attr, concepts, threshold)

        if matched_concepts:
            # Select the most specific concept (parent concept)
            closest_match = select_parent_concept([mc["concept"] for mc in matched_concepts])
            if closest_match:
                mapped_action[attr] = {
                    "closest_match": closest_match["name"],
                    "all_matches": matched_concepts,
                    "value": value
                }
            else:
                mapped_action[attr] = {
                    "closest_match": None,
                    "all_matches": matched_concepts,
                    "value": value
                }
        else:
            mapped_action[attr] = {
                "closest_match": None,
                "all_matches": [],
                "value": value
            }

    return mapped_action

In [3]:
ontology_file = "OWLs/SOMA.owl"

In [4]:
cram_action = {
        "grasp-object": "cup",
        "move-to-location": "table",
        "rotate-arm": "90 degrees",
        "unknown-attribute": "some value"
    }

In [5]:
mapped_action = map_attributes_to_ontology(cram_action, ontology_file, threshold=60)

In [6]:
print("Original CRAM Action:", cram_action)
print("Mapped CRAM Action with All Matches:")
for attr, details in mapped_action.items():
    print(f"Attribute: {attr}")
    print(f"  Closest Match: {details['closest_match']}")
    print(f"  All Matches:")
    for match in details["all_matches"]:
        print(f"    - Concept: {match['concept']['name']}, Score: {match['score']}")
    print(f"  Value: {details['value']}")
    print()

Original CRAM Action: {'grasp-object': 'cup', 'move-to-location': 'table', 'rotate-arm': '90 degrees', 'unknown-attribute': 'some value'}
Mapped CRAM Action with All Matches:
Attribute: grasp-object
  Closest Match: ShapedObject
  All Matches:
    - Concept: Object, Score: 90
    - Concept: ShapedObject, Score: 67
    - Concept: ClausalObject, Score: 64
    - Concept: CreatedObject, Score: 64
    - Concept: Set, Score: 60
  Value: cup

Attribute: move-to-location
  Closest Match: Creation
  All Matches:
    - Concept: Location, Score: 90
    - Concept: Action, Score: 75
    - Concept: Relation, Score: 68
    - Concept: Creation, Score: 68
    - Concept: Tool, Score: 68
    - Concept: MentalAction, Score: 64
    - Concept: Collection, Score: 63
    - Concept: Locomotion, Score: 63
    - Concept: Motion, Score: 60
    - Concept: Catching, Score: 60
    - Concept: Option, Score: 60
    - Concept: Set, Score: 60
  Value: table

Attribute: rotate-arm
  Closest Match: Arm
  All Matches:
    

In [7]:
onto = get_ontology(ontology_file).load()
classes = extract_classes(onto)

In [8]:
len(classes)

537

In [9]:
di = {
    'k1' : 12,
    'k2' : 12,
    'k3' : 12,
    'k4' : 12,
}
di.keys()

dict_keys(['k1', 'k2', 'k3', 'k4'])

In [10]:
classes[0:2]

[{'name': 'Affordance',
  'comment': ['A relation between an object (the bearer) and others (the triggers) that describes the disposition of the bearer to be involved in an action execution that also involves some trigger object.'],
  'label': [],
  'defined_by': ['http://www.ease-crc.org/ont/SOMA-OBJ.owl'],
  'iri': 'http://www.ease-crc.org/ont/SOMA.owl#Affordance',
  'subclasses': []},
 {'name': 'Concept',
  'comment': ['A Concept is a SocialObject, and isDefinedIn some Description; once defined, a Concept can be used in other Description(s). If a Concept isDefinedIn exactly one Description, see the LocalConcept class.\nThe classifies relation relates Concept(s) to Entity(s) at some TimeInterval'],
  'label': [locstr('Concept', 'en')],
  'defined_by': ['http://www.ease-crc.org/ont/SOMA-WF.owl',
   'ont.DUL.owl',
   'http://www.ontologydesignpatterns.org/ont/dul/DUL.owl'],
  'iri': 'http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#Concept',
  'subclasses': ['Role', 'EventType', '

## Using Semantic Similarity (Spacy)

In [11]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the spaCy model with word embeddings
nlp = spacy.load("en_core_web_md")

def compute_semantic_similarity(attribute, concept_name):
    """
    Compute the semantic similarity between an attribute and a concept name using spaCy word embeddings.
    """
    attribute_doc = nlp(attribute)
    concept_doc = nlp(concept_name)
    return attribute_doc.similarity(concept_doc)

def find_closest_matches(attribute, concepts, threshold=0.7):
    """
    Find all matches for the attribute in the list of concepts that exceed the semantic similarity threshold.
    Returns a list of matched concept dictionaries along with their similarity scores.
    """
    matched_concepts = []
    for concept in concepts:
        similarity_score = compute_semantic_similarity(attribute, concept["name"])
        if similarity_score >= threshold:
            matched_concepts.append({"concept": concept, "score": similarity_score})

    # Sort matches by similarity score (highest first)
    matched_concepts.sort(key=lambda x: x["score"], reverse=True)
    return matched_concepts

def map_attributes_to_ontology(cram_action, ontology_file, threshold=0.7):
    """
    Map the attributes of the CRAM action designator to the ontology concepts using semantic similarity.
    Returns a dictionary where each attribute is mapped to:
    - The closest match (most specific concept).
    - A list of all matches that exceed the similarity threshold.
    """
    # Load the ontology
    # graph = load_ontology(ontology_file)
    onto = get_ontology(ontology_file).load()

    # Extract all entities (classes/concepts) and their hierarchy
    # concepts = extract_concepts(graph)
    concepts = extract_classes(onto)

    # Iterate over the CRAM action designator attributes
    mapped_action = {}
    for attr, value in cram_action.items():
        # Find all matches that exceed the similarity threshold
        matched_concepts = find_closest_matches(attr, concepts, threshold)

        if matched_concepts:
            # Select the most specific concept (parent concept)
            closest_match = select_parent_concept([mc["concept"] for mc in matched_concepts])
            if closest_match:
                mapped_action[attr] = {
                    "closest_match": closest_match["name"],
                    "all_matches": matched_concepts,
                    "value": value
                }
            else:
                mapped_action[attr] = {
                    "closest_match": None,
                    "all_matches": matched_concepts,
                    "value": value
                }
        else:
            mapped_action[attr] = {
                "closest_match": None,
                "all_matches": [],
                "value": value
            }

    return mapped_action

In [12]:
cram_action = {
        "grasp-object": "cup",
        "move-to-location": "table",
        "rotate-arm": "90 degrees",
        "unknown-attribute": "some value"
    }

In [13]:
ontology_file = "OWLs/SOMA.owl"

In [14]:
mapped_action = map_attributes_to_ontology(cram_action, ontology_file, threshold=0.7)

/tmp/ipykernel_24935/4178497598.py:14: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return attribute_doc.similarity(concept_doc)


In [16]:
# Print the mapped action
print("Original CRAM Action:", cram_action)
print("Mapped CRAM Action with All Matches:")
for attr, details in mapped_action.items():
    print(f"Attribute: {attr}")
    print(f"  Closest Match: {details['closest_match']}")
    print(f"  All Matches:")
    for match in details["all_matches"]:
        print(f"    - Concept: {match['concept']['name']}, Similarity: {match['score']:.2f}")
    print(f"  Value: {details['value']}")
    print()

Original CRAM Action: {'grasp-object': 'cup', 'move-to-location': 'table', 'rotate-arm': '90 degrees', 'unknown-attribute': 'some value'}
Mapped CRAM Action with All Matches:
Attribute: grasp-object
  Closest Match: Object
  All Matches:
    - Concept: Object, Similarity: 0.70
  Value: cup

Attribute: move-to-location
  Closest Match: None
  All Matches:
  Value: table

Attribute: rotate-arm
  Closest Match: Arm
  All Matches:
    - Concept: Arm, Similarity: 0.71
    - Concept: Leg, Similarity: 0.71
  Value: 90 degrees

Attribute: unknown-attribute
  Closest Match: None
  All Matches:
  Value: some value



In [18]:
classes[0:2]

[{'name': 'Affordance',
  'comment': ['A relation between an object (the bearer) and others (the triggers) that describes the disposition of the bearer to be involved in an action execution that also involves some trigger object.'],
  'label': [],
  'defined_by': ['http://www.ease-crc.org/ont/SOMA-OBJ.owl'],
  'iri': 'http://www.ease-crc.org/ont/SOMA.owl#Affordance',
  'subclasses': []},
 {'name': 'Concept',
  'comment': ['A Concept is a SocialObject, and isDefinedIn some Description; once defined, a Concept can be used in other Description(s). If a Concept isDefinedIn exactly one Description, see the LocalConcept class.\nThe classifies relation relates Concept(s) to Entity(s) at some TimeInterval'],
  'label': [locstr('Concept', 'en')],
  'defined_by': ['http://www.ease-crc.org/ont/SOMA-WF.owl',
   'ont.DUL.owl',
   'http://www.ontologydesignpatterns.org/ont/dul/DUL.owl'],
  'iri': 'http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#Concept',
  'subclasses': ['Role', 'EventType', '

## Using LLM for semantic match (Attribute & Class)

In [40]:
import dotenv
dotenv.load_dotenv()
from openai import OpenAI
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
client = OpenAI(api_key=OPENAI_API_KEY)
import ollama

In [51]:
def get_semantic_match(attribute, classes):
    """
    Use the OpenAI ChatGPT API to find the most semantically similar class for the attribute.
    """
    # Prepare the prompt
    prompt = f"""
    You are an ontology matching assistant. Your task is to find the most semantically similar class for the attribute: "{attribute}".

    Here is a list of ontology classes with their names and descriptions:
    {classes}

    Return the name of the most semantically similar class for the attribute "{attribute}". If no suitable match is found, return "None".
    """

    # Call the OpenAI API
    response = ollama.chat(
        model="llama3.2",
        messages=[
            {"role": "system", "content": "You are an ontology matching assistant."},
            {"role": "user", "content": prompt}
        ],
        # max_tokens=100,
        # temperature=0.2  # Lower temperature for more deterministic responses
    )
    print(f"Ollama Called and the response is {response}")
    # Extract the matched class from the response
    matched_class = response["choices"][0]["message"]["content"].strip()
    return matched_class

In [47]:
def map_attributes_to_ontology(cram_action, ontology_file, threshold=0.7):
    """
    Map the attributes of the CRAM action designator to the ontology concepts using an LLM.
    Returns a dictionary where each attribute is mapped to:
    - The closest match (most semantically similar class).
    - A list of all matches that exceed the similarity threshold.
    """
    # Load the ontology
    # graph = load_ontology(ontology_file)
    onto = get_ontology(ontology_file).load()

    # Extract all entities (classes/concepts) and their hierarchy
    # concepts = extract_concepts(graph)
    concepts = extract_classes(onto)

    # Iterate over the CRAM action designator attributes
    mapped_action = {}
    for attr, value in cram_action.items():
        # Prepare the list of classes for the LLM
        classes_for_llm = "\n".join(
            [f"- Name: {concept['name']}, Description: {concept['comment'][0] if concept['comment'] else 'No description'}"
             for concept in concepts]
        )

        # Get the most semantically similar class using the LLM
        matched_class = get_semantic_match(attr, classes_for_llm)

        if matched_class and matched_class.lower() != "none":
            # Find the matched concept dictionary
            matched_concept = next((concept for concept in concepts if concept["name"] == matched_class), None)
            if matched_concept:
                mapped_action[attr] = {
                    "closest_match": matched_concept["name"],
                    "value": value
                }
            else:
                mapped_action[attr] = {
                    "closest_match": None,
                    "value": value
                }
        else:
            mapped_action[attr] = {
                "closest_match": None,
                "value": value
            }

    return mapped_action

In [48]:
cram_action = {
        "grasp-object": "cup",
        "move-to-location": "table",
        "rotate-arm": "90 degrees",
        "unknown-attribute": "some value"
    }

In [49]:
ontology_file = "OWLs/SOMA.owl"

In [50]:
mapped_action = map_attributes_to_ontology(cram_action, ontology_file)

ValidationError: 1 validation error for ChatRequest
model
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/string_too_short

In [ ]:
print("Original CRAM Action:", cram_action)
print("Mapped CRAM Action with LLM Semantic Matching:")
for attr, details in mapped_action.items():
    print(f"Attribute: {attr}")
    print(f"  Closest Match: {details['closest_match']}")
    print(f"  Value: {details['value']}")
    print()